In [1]:
import sys
sys.path.append('../scripts')
from scripts.cross_validation import loocv, k_fold_cv
from scripts.data_preprocessing import load_data, exclude_outliers
from sklearn.ensemble import RandomForestRegressor

# Load and preprocess data
X, y = load_data()

# Exclude outliers
X_filtered, y_filtered = exclude_outliers(X, y)

rf = RandomForestRegressor()

Number of outliers:  14


C:\Users\Tea\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [2]:
print('LOOCV')
loocv(X, y, rf)

print('\n10-fold')
k_fold_cv(X, y, rf, 6)

LOOCV
R squared (LOOCV): -1.77194194867121
Average MSE (LOOCV): 94.46452340425532
Average RMSE (LOOCV): 9.719286157133935
Average MAE (LOOCV): 4.474326241134753

10-fold
Mean MSE: 98.19982880434783
Mean MAE: 5.092542270531401
Mean R2: 0.06768648458699243
RMSE 9.909582675589716


In [3]:
print('LOOCV')
loocv(X_filtered, y_filtered, rf)

print('\n10-fold')
k_fold_cv(X_filtered, y_filtered, rf, 6)

LOOCV
R squared (LOOCV): -1.0194321125491226
Average MSE (LOOCV): 31.168020472440947
Average RMSE (LOOCV): 5.582832656675368
Average MAE (LOOCV): 3.5582677165354335

10-fold
Mean MSE: 39.504634740259746
Mean MAE: 4.050284992784993
Mean R2: -0.8549372806574076
RMSE 6.28527125431033
